In [1]:
#
import sys
import os
sys.path.append("/home/sergey/drclinics/common")
sys.path.append("/home/sergey/drclinics/reports")
#


import pandas as pd
from report_utils import query_by_name, send_mail
from configs import load_json_config

from connect_platform import connect_telemed
from log import log
import datetime
from utils import get_path, DATETIME_FORMAT

from universal_connection import UniversalConnection, DBType

from log import log

In [2]:
sql = """
with t1 as (
select 	a.patient_id,
		row_number() over (partition by patient.id order by end_ desc) rownum,
		concat_ws(' ', p_patient.last_name, p_patient.first_name, p_patient.middle_name) patient_fio,
		case when pc.product_id in (
            select id from product
            where full_name like '%ВЭБ%') then 'VEBMED'
        	else 'ПРОЧИЕ'
        end promo_type,
        promo.start_,
        promo.end_
from appointment a
left join promotion promo on a.promotion_id = promo.id
inner join product_condition pc
   on promo.product_condition_id = pc.id
---
left join patient on a.patient_id = patient.id
inner join person p_patient on p_patient.id = patient.person_id
---
where a.finished at time zone 'UTC' > '2020-03-30'
and a.good
  and not (
    a.patient_id in (
        select patient_id
        from patient_categories pcat
        inner join reference rf
           on rf.id=pcat.reference_id
        where rf.code='TEST'
        )
    or lower(a.report_comment)='тест'
    )
and pc.product_id in (
            select id from product
            where full_name like '%ВЭБ%')       
-----
),
-----
t1_group as (
select		patient_id,
			patient_fio,
			promo_type,
			end_
from t1 
--
where rownum = 1
),
t2 as (
select 	row_number() over (partition by patient.id order by end_ desc) rownum,
		patient.id patient_id,
		concat_ws(' ', p_patient.last_name, p_patient.first_name, p_patient.middle_name) patient_fio,
        partner."name" partner_name,
        --promo.start_,
        promo.start_,
        promo.end_
from patient
--
inner join person p_patient on p_patient.id = patient.person_id
--
inner join patient_promotion as pp on pp.patient_id = patient.id
inner join promotion promo on promo.id = pp.promotion_id 
--
left join product_condition pc
   		on promo.product_condition_id = pc.id
--
inner join appointment a on a.patient_id = patient.id 
--
inner join partner on promo.partner_id = partner.id 
--
where pc.product_id not in (
            	select id from product
            	where full_name like '%ВЭБ%')
and patient.id in (select distinct patient_id from t1)
-----
),
-----
t2_group as (
select		patient_id,
			patient_fio,
			partner_name,
			start_,
			end_
from t2 
--
where rownum = 1
and end_ > '2020-05-31'
and start_ <= '2020-05-31'
)
-----------------------------
select 	t1_group.patient_id,
		t1_group.patient_fio,
		t1_group.promo_type,
		t1_group.end_,
		t2_group.partner_name,
		t2_group.start_ partner_start,
		t2_group.end_ partner_end
from t1_group left join t2_group 
on t1_group.patient_id = t2_group.patient_id
order by t1_group.patient_id

"""

In [3]:
connection = UniversalConnection('../../../.credentials/telemed/prom.cfg', DBType.Postgres)
data = connection.query(sql)
connection.close()

2020-05-19 17:07:46 connect to postgres database using config file "../../../.credentials/telemed/prom.cfg"
2020-05-19 17:07:46 creating ssh tunnel to 172.16.100.19 as root...
2020-05-19 17:07:47 connect postgres using parameters:
                    database: telemed
				    user: norekhov
				    password: ***masked***
				    host: localhost
				    port: 34145
2020-05-19 17:07:47 @telemed: execute sql:
				    SET TIME ZONE 'Europe/Moscow'
				    None
2020-05-19 17:07:47 @telemed query:
                    with t1 as (
				    select 	a.patient_id,
				    		row_number() over (partition by patient.id order by end_ desc) rownum,
				    		concat_ws(' ', p_patient.last_name, p_patient.first_name, p_patient.middle_name) patient_fio,
				    		case when pc.product_id in (
				                select id from product
				                where full_name like '%ВЭБ%') then 'VEBMED'
				            	else 'ПРОЧИЕ'
				            end promo_type,
				            promo.start_,
				            

In [4]:
df = pd.DataFrame(data).copy()

In [5]:
df.head(15)

,patient_id,patient_fio,promo_type,end_,partner_name,partner_start,partner_end
0,1038,Новохатская Татьяна Владимировна,VEBMED,2020-05-31,None,None,None
1,1093,Жинкина Анна Михайловна,VEBMED,2020-05-31,None,None,None
2,2383,Ковалева Лариса Валерьевна,VEBMED,2020-05-31,None,None,None
3,3836,Анонимp4545 Анонимp4545,VEBMED,2020-05-31,None,None,None
4,4435,Константинов Евгений Алексеевич,VEBMED,2020-05-31,None,None,None
5,4568,Орлова Гульнара Рафаэлевна,VEBMED,2020-05-31,None,None,None
6,4984,Коньшин Александр Вадимович,VEBMED,2020-05-31,Альянс Жизнь,2020-05-01,2021-04-30
7,5117,Шкумат Людмила Борисовна,VEBMED,2020-05-31,None,None,None
8,6004,Кузнецова Виктория Викторовна,VEBMED,2020-05-31,None,None,None
9,6022,Kzdsz Kzdsz Kzdsz,VEBMED,2020-05-31,None,None,None


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20224 entries, 0 to 20223
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   patient_id     20224 non-null  int64 
 1   patient_fio    20224 non-null  object
 2   promo_type     20224 non-null  object
 3   end_           20224 non-null  object
 4   partner_name   452 non-null    object
 5   partner_start  452 non-null    object
 6   partner_end    452 non-null    object
dtypes: int64(1), object(6)
memory usage: 1.1+ MB


In [7]:
# web-patients
len(df.patient_id.unique())

20224

In [8]:
df['partner_name'].isnull().values.sum()

19772

In [9]:
len(df.patient_id.unique()) - df['partner_name'].isnull().values.sum()

452

In [10]:
d = dict()

d = {
    'Всего пациентов WEB':len(df.patient_id.unique()),
    'Пациентов с другим продуктом':len(df.patient_id.unique()) - df['partner_name'].isnull().values.sum()
    
}

In [11]:
report = pd.DataFrame.from_dict(d, orient='index').sort_index()
report.columns = ['Количество']

In [12]:
report

,Количество
Всего пациентов WEB,20224
Пациентов с другим продуктом,452


In [13]:
report_date: str = datetime.datetime.now().strftime(DATETIME_FORMAT).replace(':', '-')

In [14]:
"""
file_name = 'after_web' + report_date + '.xlsx'
writer = pd.ExcelWriter(file_name, engine='xlsxwriter')

df.to_excel(writer, sheet_name='Data', index=False)
worksheet = writer.sheets['Data']
# worksheet.set_column('A:A', 50)
worksheet.set_column(0, 0, 12)
worksheet.set_column(1, 1, 35)
worksheet.set_column(2, 3, 12)
worksheet.set_column(4, 4, 25)
worksheet.set_column(5, 5, 12)

report.to_excel(writer, sheet_name='Count')
worksheet = writer.sheets['Count']
worksheet.set_column('A:A', 35)
worksheet.set_column(1, len(report.columns), 12)


writer.save()
"""

"\nfile_name = 'after_web' + report_date + '.xlsx'\nwriter = pd.ExcelWriter(file_name, engine='xlsxwriter')\n\ndf.to_excel(writer, sheet_name='Data', index=False)\nworksheet = writer.sheets['Data']\n# worksheet.set_column('A:A', 50)\nworksheet.set_column(0, 0, 12)\nworksheet.set_column(1, 1, 35)\nworksheet.set_column(2, 3, 12)\nworksheet.set_column(4, 4, 25)\nworksheet.set_column(5, 5, 12)\n\nreport.to_excel(writer, sheet_name='Count')\nworksheet = writer.sheets['Count']\nworksheet.set_column('A:A', 35)\nworksheet.set_column(1, len(report.columns), 12)\n\n\nwriter.save()\n"

# AUTO-WIDTH

In [16]:
global writer
writer = None

In [17]:
def corr_width(x: int):
    if x <= 7:
        return 7
    elif x >=50:
        return 50
    return x

def auto_columns_width(file_name, data, sheet_name, formatting=[]):
    
    # writer
    global writer
    if not writer:
        writer = pd.ExcelWriter(file_name, engine='xlsxwriter')
    
    # drop pandas RangeIndex
    if isinstance(data.index, pd.core.indexes.range.RangeIndex):
        data.to_excel(writer, sheet_name=sheet_name, index=False)
    else:
        data.to_excel(writer, sheet_name=sheet_name)
    
    # select current worksheet
    worksheet = writer.sheets[sheet_name]
    
    # read columns
    cols = list(data.columns)
    cols_iter = list(data.columns)
    
    # datacolumns to str
    for i in range(len(cols)):
        if isinstance(cols[i], datetime.date):
            cols[i] = cols[i].strftime('%Y-%m-%d')
    
    # if index exists = set k=1
    k = 0
    if data.index.dtype == object:
        k = 1
    
    # setting index length
    worksheet.set_column(0, 0, max(data.index.astype(str).map(len)) * 1.5)
    
    # setting columns length
    for i, col in enumerate(cols, k):
        
        if data[cols_iter[i-k]].dtype == int:
            col_len = corr_width(len(col)) * 1.5
        else:
            # эта помойка выбирает что длиннее - название столбца или какое-то значение внутри него
            col_len = max(corr_width(max(data[(cols_iter[i-k])].fillna(0).astype(str).map(len))), len(col))
            col_len *= 1.5
        
        print(i, col, col_len)
        worksheet.set_column(i, i, col_len)
        

In [18]:
file_name = 'subscribe-after-web' + report_date + '.xlsx'

In [19]:
auto_columns_width(file_name, df, 'data')

0 patient_id 15.0
1 patient_fio 67.5
2 promo_type 15.0
3 end_ 15.0
4 partner_name 45.0
5 partner_start 19.5
6 partner_end 16.5


In [20]:
auto_columns_width(file_name, report, 'Quantity')

1 Количество 15.0


In [21]:
writer.save()

# END

In [22]:
# writer

In [23]:
#isinstance(df.index, pd.core.indexes.range.RangeIndex)

In [24]:
# type(df.index)

In [25]:
# report.index

In [26]:
max(df.patient_fio.map(len))

45

In [27]:
df['patient_fio'].dtype

dtype('O')

In [28]:
df['patient_id'].dtype

dtype('int64')

In [29]:
isinstance(df['promo_type'].dtype, object)

True

In [30]:
df['patient_id'].dtype == int

True

In [31]:
max(report.index.map(len))

28

In [32]:
type(report.index)

pandas.core.indexes.base.Index

In [33]:
report.index.dtype == object

True

In [34]:
report.index.dtype

dtype('O')

In [35]:
max(df['end_'].fillna(0).astype(str).map(len))

10

In [36]:
list(df.columns)

['patient_id',
 'patient_fio',
 'promo_type',
 'end_',
 'partner_name',
 'partner_start',
 'partner_end']

In [37]:
df.end_

0        2020-05-31
1        2020-05-31
2        2020-05-31
3        2020-05-31
4        2020-05-31
            ...    
20219    2020-05-31
20220    2020-05-31
20221    2020-05-31
20222    2020-05-31
20223    2020-05-31
Name: end_, Length: 20224, dtype: object

In [38]:
df['end_'].dtype

dtype('O')

In [39]:
df.columns[0]

'patient_id'

In [40]:
report.columns[0]

'Количество'